# `System.Text.Json`: preserving escape characters

The `JsonSerializer.Serialize` method produces text that preserves escape characters.

## C# by default escape characters

In [1]:
#!csharp

string s = "do not escape this: \\d";

s

do not escape this: \d

This default behavior can be a problem in the world of JSON:

In [2]:
#!csharp

using System.Text.Json;

string json =
    $$"""
    {
        "my-property": "{{s}}"
    }
    """;

JsonDocument.Parse(json)

Error: System.Text.Json.JsonReaderException: 'd' is an invalid escapable character within a JSON string. The string should be correctly escaped. LineNumber: 1 | BytePositionInLine: 41.
   at System.Text.Json.ThrowHelper.ThrowJsonReaderException(Utf8JsonReader& json, ExceptionResource resource, Byte nextByte, ReadOnlySpan`1 bytes)
   at System.Text.Json.Utf8JsonReader.ConsumeStringAndValidate(ReadOnlySpan`1 data, Int32 idx)
   at System.Text.Json.Utf8JsonReader.ConsumeString()
   at System.Text.Json.Utf8JsonReader.ConsumeValue(Byte marker)
   at System.Text.Json.Utf8JsonReader.ReadSingleSegment()
   at System.Text.Json.Utf8JsonReader.Read()
   at System.Text.Json.JsonDocument.Parse(ReadOnlySpan`1 utf8JsonSpan, JsonReaderOptions readerOptions, MetadataDb& database, StackRowStack& stack)
   at System.Text.Json.JsonDocument.Parse(ReadOnlyMemory`1 utf8Json, JsonReaderOptions readerOptions, Byte[] extraRentedArrayPoolBytes, PooledByteBufferWriter extraPooledByteBufferWriter)
   at System.Tex

To avoid this kind of `invalid escapable character within a JSON string` error, we use the `JsonSerializer.Serialize` method:

In [3]:
#!csharp

using System.Text.Json;

string json2 =
    $$"""
    {
        "my-property": {{JsonSerializer.Serialize(s)}}
    }
    """;

JsonDocument.Parse(json2)

my-property,"""do not escape this: \d"""


Notice there are no quotation marks after `"my-property":` as the JSON serializer will produce them:

In [4]:
#!csharp

JsonSerializer.Serialize(s)

"do not escape this: \\d"

## <!-- -->

[Bryan Wilhite is on LinkedIn](https://www.linkedin.com/in/wilhite)🇺🇸💼